In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler
import torch.nn.functional as F
from torch.nn.parallel import DataParallel
from torch.cuda.amp import autocast, GradScaler
readpath='/data/disk3/DataBase_stocks/AllSample/'

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(device)
else:
    device = torch.device("cpu")
    print("GPU 不可用，将在 CPU 上运行")

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:32"
torch.cuda.empty_cache()  # 释放显存

# 定义标准化函数
def normalize_row(row):
    data = row.dropna()
    mean = np.mean(data)
    std = np.std(data)
    row[row.notna()] = (row[row.notna()] - mean) / std
    return row
df=pd.read_csv(readpath+'adjopen.csv',index_col=0)
df.drop([x for x in df.columns if x[-2:]=='BJ'],axis=1,inplace=True)
dfopen = df.apply(normalize_row, axis=1)
df=pd.read_csv(readpath+'adjclose.csv',index_col=0)
df.drop([x for x in df.columns if x[-2:]=='BJ'],axis=1,inplace=True)
dfclose = df.apply(normalize_row, axis=1)
df=pd.read_csv(readpath+'adjhigh.csv',index_col=0)
df.drop([x for x in df.columns if x[-2:]=='BJ'],axis=1,inplace=True)
dfhigh = df.apply(normalize_row, axis=1)
df=pd.read_csv(readpath+'adjlow.csv',index_col=0)
df.drop([x for x in df.columns if x[-2:]=='BJ'],axis=1,inplace=True)
dflow = df.apply(normalize_row, axis=1)
df=pd.read_csv(readpath+'volume.csv',index_col=0)
df.drop([x for x in df.columns if x[-2:]=='BJ'],axis=1,inplace=True)
dfvolume = df.apply(normalize_row, axis=1)
df=pd.read_csv(readpath+'vwap_adj.csv',index_col=0)
df.drop([x for x in df.columns if x[-2:]=='BJ'],axis=1,inplace=True)
dfvwap = df.apply(normalize_row, axis=1)
dfs=[dfclose,dfopen,dfhigh,dflow,dfvwap,dfvolume]
dff=[]
for df in dfs:  #删除测试集时间段内未上市的
    df_filled = df[(df.index < 20210101) & (df.index>=20190101)]
    df_filled=df_filled.dropna(axis=1,how='all')
    df_filled = df_filled.fillna(method='bfill')
    dff.append(df_filled)

dfm=pd.concat(dff,axis=0,join='inner')
columns_to_drop = []
for column in dfm.columns:
    # 将每列转换为布尔值，True表示空值，False表示非空值
    is_null = dfm[column].isnull()
    # 使用rolling函数计算连续空值的长度
    consecutive_nulls = is_null.rolling(5, min_periods=1).sum()
    # 检查是否存在连续空值长度大于等于5的情况
    if any(consecutive_nulls >= 5):
        columns_to_drop.append(column)
for i in range(len(dff)):
    dff[i] = dff[i].drop(columns=columns_to_drop)
#再对齐一下列，不知道为什么concat后还是没对齐：
dfmm=pd.concat(dff,axis=0,join='inner')
for i in range(len(dff)):
    dff[i] = dff[i][dff[i].columns.intersection(dfmm.columns)]
time_length =487
stock_code_length = 4121
T=60
# 创建一个空的三维数组，用于存放合并后的数据
X_tensor = np.zeros((time_length, stock_code_length, 6))

# 合并6个DataFrame的数据
for i, df in enumerate(dff):
    # 将DataFrame的值复制到对应的tensor切片中
    X_tensor[:, :, i] = df.values
X_tensor=torch.Tensor(X_tensor)
total_samples = 428  # 指定总样本数量
time_steps_per_sample = 60  # 指定每个样本的时间步数
num_stocks = 4121  # 股票数量
num_features = 6  # 特征数量

# 初始化新的X_train
X_train = torch.zeros((total_samples, time_steps_per_sample, num_stocks, num_features))

# 将数据拆分成样本

for i in range(total_samples):
    end_idx = i + time_steps_per_sample
    X_train[i] = X_tensor[i:end_idx]
print(X_train.shape)

cuda
torch.Size([428, 60, 4121, 6])


torch.Size([428, 0])

In [14]:
twap=pd.read_csv('twap_all.csv',index_col=[0])
adj=pd.read_csv('adjfactor.csv',index_col=[0])
twap.columns=[x[:6] for x in twap.columns]
adj.columns=[x[:6] for x in adj.columns]
open=twap*adj
# open.columns=open.columns.astype(int)
ret=(open-open.shift(1))/open.shift(1)
ret=ret.shift(-1)
ret= ret.fillna(method='bfill')
ret=ret[(ret.index < 20210105) & (ret.index>=20190101)]
columns=[x.split('.')[0] for x in dff[0].columns]
ret= ret[ret.columns.intersection(columns)]
ret=ret.iloc[60:,]
ret
array = ret.to_numpy()  # 将DataFrame转换为NumPy数组
Y_train = np.reshape(array, ret.shape)  # 将数组reshape为张量
Y_train=torch.Tensor(Y_train)
Y_train.shape

torch.Size([428, 4121])

In [ ]:
class MyModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_factors, num_stocks):
        super(MyModel, self).__init__()

        self.num_stocks = num_stocks
        self.num_factors=num_factors
        
        # 定义LSTM层
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        
        # 定义全连接层
        self.fc = nn.Linear(hidden_size, num_factors)  #h
        
        # 定义批标准化层
        self.bn = nn.BatchNorm1d(num_factors)  #z

    def forward(self, x):
        batch_size, T, _, num_features = x.size()
        
        # 将num_stocks维度移到batch_size之后
        x = x.view(-1, T, num_features)
        
        # LSTM层
        lstm_out, _ = self.lstm(x)
        
        # 最后一个时间步的输出
        lstm_out = lstm_out[:, -1, :]
        
        # 全连接层
        fc_out = self.fc(lstm_out)
        
        # 批标准化层
        factor_output = self.bn(fc_out)
        
        # 将结果恢复成(batch_size, num_stocks, num_factors)形状
        factor_output = factor_output.view(batch_size, self.num_stocks, num_factors)
        
        return factor_output
# 数据准备：假设有num_stocks只股票，每只股票有num_features个特征
num_stocks = 4121
num_features = 6
T = 60  # 假设时间步为30
num_epochs = 8
batch_size = 4

# 创建数据，X_train的形状应为(训练集总长, T, num_stocks, num_features)，y_train的形状应为(训练集总长, num_stocks)
X_train
Y_train 

# 模型构建

# 创建模型实例
input_size = num_features
hidden_size = 64
num_factors = 1  #60个等权输出？

model = MyModel(input_size, hidden_size, num_factors, num_stocks)


# model = DataParallel(model)
model.to(device)  # 将模型移动到GPU

# 定义优化器
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# 训练模型
for epoch in range(num_epochs):
    torch.cuda.empty_cache() 
    for i in range(0, X_train.size(0)-4, batch_size):
        inputs = X_train[i:i+batch_size].to(device)
        labels = Y_train[i:i+batch_size].to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        
        # 计算损失函数
        loss = 0
        for j in range(num_stocks):
            # 对于每只股票，计算均方误差损失
            mse_loss = F.mse_loss(outputs[:, j, :], labels[:, j],reduction='mean')
            loss += mse_loss  # 或者 mae_loss，根据需求选择
            
        print(epoch,i,loss.item()/num_stocks)
        loss.backward()
        optimizer.step()

        


In [38]:
torch.save(model.state_dict(), 'model_params.pth')

In [2]:
dfftest=[]
for df in dfs:  #删除测试集时间段内未上市的
    df_filled = df[(df.index < 20211231) & (df.index>=20210101)]
    df_filled=df_filled.dropna(axis=1,how='all')
    df_filled = df_filled.fillna(method='bfill')
    dfftest.append(df_filled)
for i in range(len(dff)):
    dfftest[i] = dfftest[i][dfftest[i].columns.intersection(dfmm.columns)]
print(dfftest[0].shape)
time_length =242
stock_code_length = 4121
T=60
# 创建一个空的三维数组，用于存放合并后的数据
X_tensortest = np.zeros((time_length, stock_code_length, 6))

# 合并6个DataFrame的数据
for i, df in enumerate(dfftest):
    # 将DataFrame的值复制到对应的tensor切片中
    X_tensortest[:, :, i] = df.values
X_tensortest=torch.Tensor(X_tensortest)
total_samples = 183  # 指定总样本数量
time_steps_per_sample = 60  # 指定每个样本的时间步数
num_stocks = 4121  # 股票数量
num_features = 6  # 特征数量

# 初始化新的X_train
X_test = torch.zeros((total_samples, time_steps_per_sample, num_stocks, num_features))

# 将数据拆分成样本

for i in range(total_samples):
    end_idx = i + time_steps_per_sample
    X_test[i] = X_tensortest[i:end_idx]

close=pd.read_csv('adjclose.csv',index_col=0)
close.drop([x for x in close.columns if x[-2:]=='BJ'],axis=1,inplace=True)
ret=(close-close.shift(1))/close.shift(1)
ret= ret.fillna(method='bfill')
ret=ret[(ret.index <= 20211231) & (ret.index>=20210101)]
ret= ret[ret.columns.intersection(dff[0].columns)]
ret=ret.iloc[T:,]
array = ret.to_numpy()  # 将DataFrame转换为NumPy数组
Y_test = np.reshape(array, ret.shape)  # 将数组reshape为张量
Y_test=torch.Tensor(Y_test)

(242, 4121)


In [6]:
X_test.shape

torch.Size([183, 60, 4121, 6])

In [7]:
class MyModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_factors, num_stocks):
        super(MyModel, self).__init__()

        self.num_stocks = num_stocks
        self.num_factors= num_factors
        
        # 定义LSTM层
        self.lstm = nn.LSTM(input_size, hidden_size,num_layers=2, batch_first=True)
        
        # 定义全连接层
        self.fc = nn.Linear(hidden_size, num_factors)  #h
        
        # 定义批标准化层
        self.bn = nn.BatchNorm1d(num_factors)  #z

    def forward(self, x):
        batch_size, T, _, num_features = x.size()
        
        # 将num_stocks维度移到batch_size之后
        x = x.view(-1, T, num_features)
        
        # LSTM层
        lstm_out, _ = self.lstm(x)
        
        # 最后一个时间步的输出
        lstm_out = lstm_out[:, -1, :]
        
        # 全连接层
        fc_out = self.fc(lstm_out)
        
        # 批标准化层
        factor_output = self.bn(fc_out)
        
        # 将结果恢复成(batch_size, num_stocks, num_factors)形状
        factor_output = factor_output.view(batch_size, self.num_stocks, num_factors)
        
        return factor_output



In [9]:
from scipy.stats import spearmanr
from scipy.stats import pearsonr
torch.cuda.empty_cache()
#input_size, hidden_size, num_factors, num_stocks
model = MyModel(6,64,60,4121)
# 加载保存的模型参数
model.load_state_dict(torch.load('model_params_long2.pth'))

# 设置模型为评估模式
model.eval()
ic_values = []

with torch.no_grad():
    input_sample = X_test  # 获取单个样本并添加批次维度
    label_sample = Y_test
    # 使用模型进行推理并获取单个样本的预测输出（c值）
    factors = model(input_sample)
    c_value=factors.mean(dim=2)
    print(c_value.shape)
    print(Y_test.shape)
    corr1=[]
    corr2=[]
    for i in range(c_value.shape[0]): 
        # 创建遮罩以识别c_value和Y_test中的非NaN值
        mask_c = ~torch.isnan(c_value[i])
        mask_y = ~torch.isnan(Y_test[i])
        
        # 组合这些遮罩以获取c_value和Y_test的共同遮罩
        mask = mask_c & mask_y

        # 将遮罩应用于张量
        c_value_filtered = c_value[i][mask]
        Y_test_filtered = Y_test[i][mask]

    
        # 计算每个输入的均值
        mean_c = torch.mean(c_value_filtered)
        mean_y = torch.mean(Y_test_filtered)

        # 计算皮尔逊相关系数的分子和分母
        numerator = torch.sum((c_value_filtered - mean_c) * (Y_test_filtered - mean_y))
        denominator_c = torch.sqrt(torch.sum((c_value_filtered - mean_c)**2))
        denominator_y = torch.sqrt(torch.sum((Y_test_filtered - mean_y)**2))

        # 计算皮尔逊相关系数
        pearson_corr = numerator / (denominator_c * denominator_y)
        corr1.append(pearson_corr.item())
        # 计算秩次差值
        rank_X = torch.argsort(c_value[i].reshape(-1))
        rank_Y = torch.argsort(Y_test[i].reshape(-1))
        differences = rank_X - rank_Y
        # 计算斯皮尔曼秩相关系数
        n = len(c_value[i])
        spearman_corr = 1 - (6 * torch.sum(differences**2)) / (n * (n**2 - 1))
        corr2.append(spearman_corr)
    corr1=np.mean(corr1)
    corr2=np.mean(corr2)
    print('Pearson:',corr1)
    print('Spearman:',corr2)

RuntimeError: [enforce fail at alloc_cpu.cpp:75] err == 0. DefaultCPUAllocator: can't allocate memory: you tried to allocate 225687875584 bytes. Error code 12 (Cannot allocate memory)

In [43]:
from scipy.stats import spearmanr
from scipy.stats import pearsonr
# 将测试集数据（X_test）和标签（Y_test）移动到GPU（如果模型在GPU上）
# 存储每个样本的IC值
ic_values = []

with torch.no_grad():
    for i in range(X_test.size(0)):
        input_sample = X_test[i].unsqueeze(0)  # 获取单个样本并添加批次维度
        label_sample = Y_test[i]
        # 使用模型进行推理并获取单个样本的预测输出（c值）
        c_value = model(input_sample)

        # 将预测结果与实际标签转换为NumPy数组
        c_value = c_value.squeeze().cpu().numpy()
        label_sample = label_sample.cpu().numpy()
        c_value=np.nan_to_num(c_value, nan=0)
        label_sample=np.nan_to_num(label_sample, nan=0)
        # 计算单个样本的IC值并添加到列表中
        ic_sample, _ = pearsonr(label_sample, c_value)
        ic_values.append(ic_sample)
print(np.mean(ic_values))

0.00042065963876592013


In [40]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler
import torch.nn.functional as F

In [ ]:
class MyModel(nn.Module):
    ###input_size:输入的特征维度  hidden_size:隐藏层神经元个数

    def __init__(self, input_size, hidden_size, output_size):
        super(MyModel, self).__init__()
        
        # 定义LSTM层
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        
        # 定义全连接层
        self.fc = nn.Linear(hidden_size, output_size)
        
        # 定义批标准化层
        self.bn = nn.BatchNorm1d(output_size)
        
    def forward(self, x):
        # LSTM层
        lstm_out, _ = self.lstm(x)
        
        # 最后一个时间步的输出
        lstm_out = lstm_out[:, -1, :]
        
        # 全连接层
        fc_out = self.fc(lstm_out)
        
        # 批标准化层
        factor_output = self.bn(fc_out)
        
        return factor_output


In [ ]:
# 自定义损失函数
def custom_loss(y_true, y_pred):
    # 计算损失函数，可以根据实际需求进行调整
    loss = torch.mean((y_true - y_pred) ** 2)
    
    # 计算标准化因子相关性矩阵的L2范数
    factor_corr_matrix = torch.matmul(torch.transpose(y_pred, 0, 1), y_pred)
    l2_penalty = torch.norm(factor_corr_matrix, p='fro')
    
    # 添加相关性惩罚项
    loss += lambda_penalty * l2_penalty
    
    return loss

# 定义相关性惩罚项的权重
lambda_penalty = 0.001  # 根据实际需求进行调整


In [ ]:
# 创建模型实例
input_size = 6  # 输入特征的数量
hidden_size = 64  # LSTM隐藏层的单元数
output_size = 1  # 多元因子的数量,先只预测收益率

model = MyModel(input_size, hidden_size, output_size)

# 定义优化器
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 定义训练数据和标签的Tensor
X_train = torch.Tensor()  # 训练数据，形状为 (样本数量, T, num_features)(,60,)
y_train = torch.Tensor()  # 标签，形状为 (样本数量, output_size)

# 训练模型
num_epochs = 100  # 根据实际需求进行调整
batch_size = 32  # 根据实际需求进行调整

for epoch in range(num_epochs):
    print('epoch:',epoch)
    for i in range(0, X_train.size(0), batch_size):
        inputs = X_train[i:i+batch_size]
        labels = y_train[i:i+batch_size]
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = custom_loss(labels, outputs)
        loss.backward()
        optimizer.step()


In [ ]:
class MyModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_factors, num_stocks):
        super(MyModel, self).__init__()

        self.num_stocks = num_stocks
        
        # 定义LSTM层
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        
        # 定义全连接层
        self.fc = nn.Linear(hidden_size, num_factors)
        
        # 定义批标准化层
        self.bn = nn.BatchNorm1d(num_factors)
        
    def forward(self, x):
        batch_size, T, _, num_features = x.size()
        
        # 将num_stocks维度移到batch_size之后
        x = x.view(-1, T, num_features)
        
        # LSTM层
        lstm_out, _ = self.lstm(x)
        
        # 最后一个时间步的输出
        lstm_out = lstm_out[:, -1, :]
        
        # 全连接层
        fc_out = self.fc(lstm_out)
        
        # 批标准化层
        factor_output = self.bn(fc_out)
        
        # 将结果恢复成(batch_size, num_stocks, num_factors)形状
        factor_output = factor_output.view(batch_size, self.num_stocks, -1)
        
        return factor_output


In [ ]:


# 数据准备：假设有num_stocks只股票，每只股票有num_features个特征
num_stocks = 5000
num_features = 6
T = 60  # 假设时间步为30
num_epochs = 100
batch_size = 32

# 创建数据，X_train的形状应为(训练集总长, T, num_stocks, num_features)，y_train的形状应为(训练集总长, num_stocks)
X_train = torch.Tensor(...)  # 根据实际数据填充
y_train = torch.Tensor(...)  # 根据实际数据填充

# 模型构建

# 创建模型实例
input_size = num_features
hidden_size = 64
num_factors = 10

model = MyModel(input_size, hidden_size, num_factors, num_stocks)

# 定义优化器
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练模型
for epoch in range(num_epochs):
    for i in range(0, X_train.size(0), batch_size):
        inputs = X_train[i:i+batch_size]
        labels = y_train[i:i+batch_size]
        
        optimizer.zero_grad()
        outputs = model(inputs)
        
        # 计算损失函数
        loss = 0
        for j in range(num_stocks):
            loss += custom_loss(labels[:, j], outputs[:, j, :])
        
        print(epoch,loss)
        loss.backward()
        optimizer.step()


In [20]:
torch.cuda.empty_cache()  # 释放显存
torch.cuda.empty_cache()
torch.cuda.empty_cache()
torch.cuda.empty_cache()
torch.cuda.empty_cache()
